# SmartPath: Intelligent Multimodal Acquisition for Histopathological Whole Slide Scanning

Quick Instructions:  
How to run a cell (section)?
Click blank space in front of the '...' below each section and hit Shift+Return, or click the "run the selected cells and advance" botton in the top menu

## 1. System configuration
Note: wait for "System configured!" Message

In [1]:
import os, glob, shutil, sys, copy, time, json, copy, subprocess, math, warnings
from IPython import display
from tqdm import tqdm
from pycromanager import Acquisition, Bridge, Dataset, multi_d_acquisition_events
from skimage import io, img_as_ubyte, img_as_float, img_as_uint, color, transform, exposure
from skimage.filters import threshold_mean, sobel
from skimage.measure import shannon_entropy
from skimage.util import view_as_windows, crop
import imagej
import json
import threading
import time
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.stats import norm
import scipy as sp
from shapely.geometry import mapping, shape
from tkinter import filedialog
from IPython.display import Audio 

from acquisitions import *
# from acquisitions_with_commentedout_DCC_line101 import *
from image_utils import *
from enhancer import Enhancer
from predictor import Predictor

os.environ['_JAVA_OPTIONS']="-Xmx12g"
warnings.filterwarnings('ignore')
Image.MAX_IMAGE_PIXELS = None

def config_sys(config):
    if config["exposure-level"]=="low":
        config["lsm-scan-rate"] = '500000.0000'
        config["lsm-pc-power"] = 0.3
        config["lsm-pmt-gain"] = 0.35
    if config["exposure-level"]=="mid":
        config["lsm-scan-rate"] = '400000.0000'
        config["lsm-pc-power"] = 0.4
        config["lsm-pmt-gain"] = 0.4
    if config["exposure-level"]=="high":
        config["lsm-scan-rate"] = '250000.0000'
        config["lsm-pc-power"] = 0.425
        config["lsm-pmt-gain"] = 0.425
    if config["exposure-level"]=="extreme":
        config["lsm-scan-rate"] = '200000.0000'
        config["lsm-pc-power"] = 0.45
        config["lsm-pmt-gain"] = 0.45
    config["pixel-size-shg"] = config["pixel-size-shg-base"] * 256 / config["lsm-resolution"]
    if config["enhancement-type"] is not None:
        config["enhancer"] = Enhancer(config)
    if config["classifier"] is not None:
        config["predictor"] = Predictor(config)
    return config

def distance(pos, support_points):
    pos = np.array(pos)
    support_points = np.array(support_points)
    distances = np.sqrt((pos - support_points)[:, 0]**2 + (pos - support_points)[:, 1]**2)
    idx = np.argmin(distances, axis=0)
    return idx, distances[idx]

def whole_slide_scan(config, core=None, save_path=None, acq_name=None, position_list=None, mag='4x', mda=False, z_stack=False, z_center=None, 
                     sample_depth=20, z_step=4, estimate_background=False, background_image=None, focus_dive=False):
    if mda == True:
        if position_list.shape[1] == 3:
            if z_stack:
                with Acquisition(save_path, acq_name, lsm_process_fn(config)) as acq:
                    events = multi_d_acquisition_events(xyz_positions=position_list.reshape(-1, 3), z_start=-int(sample_depth/2), z_end=int(sample_depth/2), z_step=z_step)
                    acq.acquire(events)      
            else:
                with Acquisition(save_path, acq_name) as acq:
                    events = multi_d_acquisition_events(xyz_positions=position_list.reshape(-1, 3))
                    acq.acquire(events)
        else:
            if z_center is None:
                z_center = config["Z-stage-laser"]
            if z_stack:
                with Acquisition(save_path, acq_name) as acq:
                    events = multi_d_acquisition_events(xy_positions=position_list.reshape(-1, 2), z_start=-int(sample_depth/2) + z_center, z_end=int(sample_depth/2) + z_center, z_step=z_step)
                    acq.acquire(events)
            else:
                with Acquisition(save_path, acq_name) as acq:
                    events = multi_d_acquisition_events(xy_positions=position_list.reshape(-1, 2))
                    acq.acquire(events)
    else:
        fig = plt.figure(figsize=(8, 6))
        plt.axis("off")
        show = plt.imshow(np.zeros((config["camera-resolution"][1], config["camera-resolution"][0])))
        acq_id = len(glob.glob(os.path.join(save_path, acq_name+"*")))
        acq_path = os.path.join(save_path, acq_name+"_{}".format(acq_id+1))
        os.makedirs(acq_path, exist_ok=True)
        bg_flag = False
        if estimate_background:
            bg_stack = []
        if background_image is not None and not estimate_background:
            bg_img = white_balance(copy.deepcopy(background_image), copy.deepcopy(background_image))
            
        if mag == '4x':
            pos_z = config["Z-stage-4x"]
        elif mag == '20x':
            pos_z = config["Z-stage-20x"] 
        support_points = [(99999999, 99999999)] # dummy support point
        support_focus = [pos_z]
        
        if position_list.shape[1] == 3:
            tile_count = 0
            z_positions=np.ones(position_list.shape[0]) * core.get_position()
            core.set_focus_device(config["focus-device"])
            autofocus_count = 0
            for pos in range(position_list.shape[0]):
                z_pos = position_list[pos, 2]
                x_pos = position_list[pos, 0]
                y_pos = position_list[pos, 1]
                x_pos, y_pos, z_pos = limit_stage(config, (x_pos, y_pos, z_pos)) #, (config["hard-limit-x"][0], config["hard-limit-x"][1], config["Z-stage-4x"]))
                core.set_position(z_pos)
                core.set_xy_position(x_pos, y_pos)
                xy_device = core.get_xy_stage_device()
                z_device = core.get_focus_device()
                core.wait_for_device(xy_device)
                core.wait_for_device(z_device)
                
                if focus_dive and mag=='4x':
                    support_distance = config["pixel-size-bf-4x"] * config["camera-resolution"][1] * config["autofocus-speed"]
                    idx, min_distance = distance((x_pos, y_pos), support_points)
                    if min_distance <= support_distance:
                        pos_z = support_focus[idx]
                        pos_z = limit_stage(config, (pos_z,), (config["Z-stage-4x"],))
                        core.set_position(pos_z)
                        core.wait_for_device(z_device)
                        pixels = snap_image(core, rgb=True, flip_channel=True)
                        bg_flag = is_background(pixels, t=0.35, tt=0.35)
                    else:
                        pos_z, pixels, bg_flag = autofocus(config, core, mag='4x', rgb=True, search_range=100, steps=3, snap=True, preset=z_pos) # snap at top but return center z
                        if bg_flag:
                            if len(support_points)>=2:
                                core.set_position(pos_z)
                            else:
                                pos_z = z_pos
                            core.wait_for_device(z_device)
                            pixels = snap_image(core, rgb=True, flip_channel=True)
                        else:
                            support_points.append((x_pos, y_pos))
                            support_focus.append(pos_z)
                    z_positions[pos] = pos_z
                           
                if focus_dive and mag=='20x':
                    support_distance = config["pixel-size-bf-20x"] * config["camera-resolution"][1] * config["autofocus-speed"]
                    idx, min_distance = distance((x_pos, y_pos), support_points)
                    if min_distance <= support_distance:
                        pos_z = support_focus[idx]
                        pos_z = limit_stage(config, (pos_z,), (config["Z-stage-20x"],))
                        core.set_position(pos_z)
                        core.wait_for_device(z_device)
                        pixels = snap_image(core, rgb=True, flip_channel=True)
                        bg_flag = is_background(pixels, t=0.35, tt=0.35)
                    else:
                        pos_z, pixels, bg_flag = autofocus(config, core, mag='20x', rgb=True, search_range=50, steps=3, snap=True, preset=z_pos) # snap at top but return center z
                        if bg_flag:
                            if len(support_points)>=2:
                                core.set_position(pos_z)
                            else:
                                pos_z = z_pos
                            core.wait_for_device(z_device)
                            pixels = snap_image(core, rgb=True, flip_channel=True)
                        else:
                            pos_z, pixels, _ = autofocus(config, core, mag='20x', rgb=True, search_range=10, steps=3, snap=True, check_background=False) # snap at top but return center z
                            support_points.append((x_pos, y_pos))
                            support_focus.append(pos_z)
                    z_positions[pos] = pos_z 
                    
                pixels = img_as_float(pixels)   
                
                if estimate_background:
                    if focus_dive:
                        bg_flag = bg_flag
                    else:
                        bg_flag = is_background(pixels, t=0.35, tt=0.35)
                        print('hard check')
                    if bg_flag:
                        print(' (background tile)')
                        redive_flag=True
                        bg_stack.append(pixels)
                    else:
                        redive_flag=False                
                if background_image is not None and not estimate_background:
                    pixels = white_balance(config, pixels, background_image)
                    pixels = flat_field(pixels, bg_img)
                    
                show.set_data(pixels)
                display.display(plt.gcf())
                display.clear_output(wait=True)
                io.imsave(acq_path+'/{}-{}.tiff'.format(pos, bg_flag), img_as_ubyte(pixels))
                tile_count = tile_count + 1
                sys.stdout.write('\r {}/{} tiles done'.format(tile_count, position_list.shape[0]))
            
        if position_list.shape[1] == 2:
            tile_count = 0
            core.set_focus_device(config["focus-device"])
            z_positions=np.ones(position_list.shape[0]) * core.get_position()
            autofocus_count = 0
            for pos in range(position_list.shape[0]):
                x_pos = position_list[pos, 0]
                y_pos = position_list[pos, 1]
                
                x_pos, y_pos = limit_stage(config, (x_pos, y_pos)) #, (config["hard-limit-x"][0], config["hard-limit-x"][1]))
                    
                xy_device = core.get_xy_stage_device()
                z_device = core.get_focus_device()
                core.set_xy_position(x_pos, y_pos)
                core.wait_for_device(xy_device)
                
                    
                if focus_dive and mag=='4x':
                    support_distance = config["pixel-size-bf-4x"] * config["camera-resolution"][1] * config["autofocus-speed"]
                    idx, min_distance = distance((x_pos, y_pos), support_points)
                    if min_distance <= support_distance:
                        pos_z = support_focus[idx]
                        pos_z = limit_stage(config, (pos_z,), (config["Z-stage-4x"],))
                        core.set_position(pos_z)
                        core.wait_for_device(z_device)
                        pixels = snap_image(core, rgb=True, flip_channel=True)
                        bg_flag = is_background(pixels, t=0.35, tt=0.35)
                    else:
                        pos_z, pixels, bg_flag = autofocus(config, core, mag='4x', rgb=True, search_range=100, steps=3, snap=True) # snap at top but return center z
                        if not bg_flag:
                            support_points.append((x_pos, y_pos))
                            support_focus.append(pos_z)
                    z_positions[pos] = pos_z
                    
                if focus_dive and mag=='20x':
                    support_distance = config["pixel-size-bf-20x"] * config["camera-resolution"][1] * config["autofocus-speed"]
                    idx, min_distance = distance((x_pos, y_pos), support_points)
                    if min_distance <= support_distance:
                        pos_z = support_focus[idx]
                        pos_z = limit_stage(config, (pos_z,), (config["Z-stage-20x"],))
                        core.set_position(pos_z)
                        core.wait_for_device(z_device)
                        pixels = snap_image(core, rgb=True, flip_channel=True)
                        bg_flag = is_background(pixels, t=0.35, tt=0.35)
                    else:
                        pos_z, pixels, bg_flag = autofocus(config, core, mag='20x', rgb=True, search_range=50, steps=3, snap=False) # snap at top but return center z
                        pos_z, pixels, bg_flag = autofocus(config, core, mag='20x', rgb=True, search_range=10, steps=3, snap=True) # snap at top but return center z
                        if not bg_flag:
                            support_points.append((x_pos, y_pos))
                            support_focus.append(pos_z)
                    z_positions[pos] = pos_z    
                    
                pixels = img_as_float(pixels)                   
                
                if estimate_background:
                    if focus_dive:
                        bg_flag = bg_flag
                    else:
                        bg_flag = is_background(pixels, t=0.35, tt=0.35)
                        print('hard check')
                    if bg_flag:
                        print(' (background tile)')
                        redive_flag=True
                        bg_stack.append(pixels)
                    else:
                        redive_flag=False                
                if background_image is not None and not estimate_background:
                    pixels = white_balance(config, pixels, background_image)
                    pixels = flat_field(pixels, bg_img)
                    
                show.set_data(pixels)
                display.display(plt.gcf())
                display.clear_output(wait=True)
                io.imsave(acq_path+'/{}-{}.tiff'.format(pos, bg_flag), img_as_ubyte(pixels))
                tile_count = tile_count + 1
                sys.stdout.write('\r {}/{} tiles done'.format(tile_count, position_list.shape[0]))
        returns = []
        if estimate_background:
            if len(bg_stack)==0:
                returns.append(background_image)
                io.imsave(acq_path+'/bg_img.tiff', img_as_ubyte(background_image))
            else:
                bg_stack= np.stack(bg_stack)
                median = np.median(bg_stack, axis=0)
                median = img_as_float(median)
                returns.append(median)
                io.imsave(acq_path+'/bg_img.tiff', img_as_ubyte(median))
        if focus_dive:
            z_positions = z_positions.reshape(position_list.shape[0], 1)
            returns.append(z_positions)
        return tuple(returns)

def autofocus(config, core, method='edge', mag='4x', interpolation='quadratic', rgb=True, search_range=45, steps=3, snap=True, crop_ratio=1.0, flip_channel=True, check_background=True, offset=0, preset=None):
    if mag=='4x':
        drift_origin = config["Z-stage-4x"]
    if mag=='20x':
        drift_origin = config["Z-stage-20x"]
    core.set_focus_device(config["focus-device"])   
    current_z = core.get_position()
    interval_z = search_range/steps
    scores = []
    positions = []
    count = 0
    for step in range(-int(np.floor(steps/2)), int(np.ceil(steps/2))):
        position_z = step * interval_z + current_z
        position_z = limit_stage(config, (position_z,), (drift_origin,))
        core.set_position(position_z)
        core.wait_for_system()
        count = count + 1
        pixels = snap_image(core, rgb=rgb, flip_channel=True)
        if check_background and step==-int(np.floor(steps/2)):
            bg_flag = is_background(pixels, t=0.35, tt=0.35)
            if bg_flag:
                if preset is not None:
                    preset = limit_stage(config, (preset,), (drift_origin,))
                    core.set_position(preset)
                else:
                    drift_origin = limit_stage(config, (drift_origin,), (drift_origin,))
                    core.set_position(drift_origin)
                core.wait_for_system()
                print("Is background")
                return drift_origin, pixels, bg_flag # TODO: return center z instead of top
        img_gray = color.rgb2gray(pixels)
        sys.stdout.write("\r Diving focus at " + str(step))
        if method == 'entropy':
            score = shannon_entropy(img_gray)
        if method == 'edge':
            score = np.mean(sobel(img_gray))
        scores.append(score)
        positions.append(position_z)
        print('Score: {}, Position {}'.format(score, position_z))
    scores_array = np.asarray(scores)
    positions_array = np.asarray(positions) 
    new_length = len(positions) * 100
    new_x = np.linspace(positions_array.min(), positions_array.max(), new_length)
    new_y = sp.interpolate.interp1d(positions_array, scores_array, kind=interpolation)(new_x)
    idx = np.argmax(new_y)
    focus_z = new_x[idx]
    focus_z = limit_stage(config, (focus_z,), (position_z,))
    if np.abs(focus_z-drift_origin) > 200:
        print("Large change in z-stage , reset focus")
        focus_z = drift_origin
        core.set_position(drift_origin)
        core.wait_for_system()
    else:
        core.set_position(focus_z)
        core.wait_for_system()
    if snap:
        pixels = snap_image(core, rgb=rgb, flip_channel=True)
        return focus_z+offset, pixels, False
    else:
        return focus_z+offset, None, False   

print("System configured!")

System configured!


In [2]:
def limit_stage(config, args=None, default=None):
    if len(args)==1:
        if default[0] < config["hard-limit-z"][0] or default[0] > config["hard-limit-z"][1]:
            raise SystemExit("Default z out of range")
        if args[0] < config["hard-limit-z"][0] or args[0] > config["hard-limit-z"][1]:
            print("Warning: z-stage out of range {}".format(args))
            if len(default)==1:
                return default[0]
            else:
                raise SystemExit("Stop acquisition")
        else:
            return args[0]
    if len(args)==2:
        if args[0] < config["hard-limit-x"][0] or args[0] > config["hard-limit-x"][1] or args[1] < config["hard-limit-y"][0] or args[1] > config["hard-limit-y"][1]:
            print("Warning: xy-stage out of range x: {} y: {}".format(args[0], args[1]))
            if len(default)==2:
                return default
            else:
                raise SystemExit("Stop acquisition")
                return None
        else:
            return args
    if len(args)==3:
        if args[0] < config["hard-limit-x"][0] or args[0] > config["hard-limit-x"][1] or args[1] < config["hard-limit-y"][0] or args[1] > config["hard-limit-y"][1] or args[2] < config["hard-limit-z"][0] or args[2] > config["hard-limit-z"][1]:
            print("Warning: stage out of range x: {} y: {} z: {}".format(args[0], args[1], args[2]))
            if len(default)==3:
                return default
            else:
                raise SystemExit("Stop acquisition")
                return None
        else:
            return args

## 2. Acquisition hardware configuration
Edit user configuration. After editing, click blank space in front of the section and hit Shift+Return, or click the "run the selected cells and advance" botton in the top menu.

In [3]:
user_config = {
    
    ### User configuration ###
    
    # Quick configuration group for LSM
    "exposure-level" : 'high',
    # 'low'     -> scan rate: '500000.0000', pockel cell gain: 0.3, PMT gain: 0.35
    # 'mid'     -> scan rate: '400000.0000', pockel cell gain: 0.4, PMT gain: 0.4
    # 'high'    -> scan rate: '250000.0000', pockel cell gain: 0.425, PMT gain: 0.425
    # 'extreme' -> scan rate: '200000.0000', pockel cell gain: 0.45, PMT gain: 0.45
    
    "snr-level" : 'low',
    # Estimiated correction according to sample SNR level. Available value: 'low', 'mid', 'high'
    
    "autofocus-speed" : 6,
    # Speed of software autofocus, integer: 1~5. Bigger value leads to faster brightfield scan but potentially lower autofocus performance
       
    "lsm-resolution" : 256, 
    # LSM scan resolution, available resolution: 256, 512, 1024
    
    "lsm-bin-factor" : 4,
    # LSM scan pixel average factor, positive integer
    
    "lsm-scan-rate" : '400000.0000', 
    # LSM scan rate, available value (string): '125000.0000', '200000.0000', '250000.0000','400000.0000', '500000.0000', '625000.0000', '1000000.0000'
    
    "lsm-pc-power" : 0.4, 
    # LSM pockel cell gain, float point value: 0.0 ~ 1.0
    
    "lsm-pmt-gain" : 0.4,
    # LSM PMT gain, float point value: 0.0 ~ 1.0
    
    "slide-box" : (-100, 600, 25500.0, 17000.0), 
    # Pre-define scan area (read out values from the stage): (start x stage position, start y stage position, end x stage position, end y stage position)
    
    "enhancement-type" : None,
    # Runtime enhancement method, available value (string or None): 'Self', 'Supervised', None
    
     "classifier" : None,
    # Automatic target detection model, available value (string or None): 'MIL', 'Supervised', None 
    
    "classifier-backbone" : "ResNet18",
    # Backbone for CNN model, available value (string): 'ResNet18', 'ResNet34' 
    
    "classifier-num-class" : 2,
    # Number of class for the detector, non-zero integer
    
    "mil-classifier-thresh" : 2,
    # Threshold for MIL classifier. Obtained from training.
    
    "slide-type" : "TMA",
    # Type of image the inference is applied, available value (string): "TMA", "slide"
    
    "gpu" : False,
    # Is GPU available? Available value (boolean): True, False       
}

### Generate acquisition configuration
Run the below section. Wait for message containing the configuration specs.

In [4]:
### right general save range check function
hard_config = {
    ### Hard configuration, 
    "pixel-size-bf-20x" : 0.222, # 0.222 micron/pixel at (1392, 1040)
    "pixel-size-bf-4x" : 1.105, # 1.305 micron/pixel at (1392, 1040)
    "pixel-size-shg-base" : 0.509, # 0.509 micron/pixel at 256
    "pixel-size-shg" : 0.509,
    "camera-resolution" : (1392, 1040), # (width, height)
    "lsm-resolution-base" : (512, 512),
    "slide-size" : (40000.0, 20000.0), # (width, height) (70000, -20000)
#     "Z-stage-20x" : -6930, # -6930 + 290 / 10500
    "Z-stage-20x" : -6980, # -6930 + 290 / 10500
    "Z-stage-laser" : -6640, #-6640 
    "Z-stage-4x" : 3570, # -2300
    "F-stage-20x" : -15800, # 11000
    "F-stage-laser" : -18500, # -17500
    "F-stage-4x" : -1000,
    "Z-bf-offset" : -10500,
    "hard-limit-z" : (-7700.0, 17000.0),
    "hard-limit-x" : (-3000.0, 40000.0),
    "hard-limit-y" : (-2200, 19000.0),
    "hard-limit-f" : (-19000, 0),
    "20x-bf-offset" : (-600, 10), # 4x + this value to 20x // (-590, 74)
    "shg-offset" : (-580, -280), # 4x + this value to shg // (-580, -172)
    "led-4x" : 4,
    "led-20x" : 5,
    "focus-device" : 'ZStage:Z:32',
    "condensor-device" : 'ZStage:F:32',
    "led-device" : ('LED-Dev1ao0', 'Voltage'),
    "obj-device" : ('Turret:O:35', 'Label'),
}
config = {**user_config, **hard_config}
config = config_sys(config)
print("Configuration specs:")
config

Configuration specs:


{'exposure-level': 'high',
 'snr-level': 'low',
 'autofocus-speed': 6,
 'lsm-resolution': 256,
 'lsm-bin-factor': 4,
 'lsm-scan-rate': '250000.0000',
 'lsm-pc-power': 0.425,
 'lsm-pmt-gain': 0.425,
 'slide-box': (-100, 600, 25500.0, 17000.0),
 'enhancement-type': None,
 'classifier': None,
 'classifier-backbone': 'ResNet18',
 'classifier-num-class': 2,
 'mil-classifier-thresh': 2,
 'slide-type': 'TMA',
 'gpu': False,
 'pixel-size-bf-20x': 0.222,
 'pixel-size-bf-4x': 1.105,
 'pixel-size-shg-base': 0.509,
 'pixel-size-shg': 0.509,
 'camera-resolution': (1392, 1040),
 'lsm-resolution-base': (512, 512),
 'slide-size': (40000.0, 20000.0),
 'Z-stage-20x': -6980,
 'Z-stage-laser': -6640,
 'Z-stage-4x': 3570,
 'F-stage-20x': -15800,
 'F-stage-laser': -18500,
 'F-stage-4x': -1000,
 'Z-bf-offset': -10500,
 'hard-limit-z': (-7700.0, 17000.0),
 'hard-limit-x': (-3000.0, 40000.0),
 'hard-limit-y': (-2200, 19000.0),
 'hard-limit-f': (-19000, 0),
 '20x-bf-offset': (-600, 10),
 'shg-offset': (-580, -2

## 3. Configure Pycro-Manager and PyImageJ
Make sure Micro-Manager and OpenScan is running appropriately. Run the below section. Wait for message "Succeeded!". Run only once, unless the notebook or Micro-Manager had restarted.

In [5]:
ij = imagej.init('fiji\\fiji\\Fiji.app')

## 4. Configure acquistion parameters
Enter values and run the below section.

In [19]:
save_path = 'data/acquisition'
# Data save path, relative path by default

acq_name = 'PA-961e-A2-20210804' # do not include space
# Acquisition name

# user_config["slide-box"] = (-3000, -500, 32500, 17000)
user_config["slide-box"] = (3000, 0, 6000, 4000)

In [20]:
acq_name_4x = acq_name + '-4x-bf'
config = {**user_config, **hard_config}
config = config_sys(config)
position_list = generate_grid(config, mag='4x')

In [21]:
default_bg = io.imread(glob.glob(os.path.join('data', 'acquisition', acq_name+"*", "bg_img.tiff"))[-1])
stitching(config, ij, save_path=save_path, acq_name=acq_name_4x, mda=False, position_list=position_list.reshape(position_list.shape[0]*position_list.shape[1], -1), flip_y=True, correction=True, background_image=default_bg)
#stitching(config, ij, save_path=save_path, acq_name=acq_name_4x, mda=False, position_list=position_list.reshape(position_list.shape[0]*position_list.shape[1], -1), flip_y=True, correction=True, background_image=bg_image)
export_slide(mag='4x')
print('\r, 4x brighfield acquisition done!')
Audio('C:/Windows/Media/Windows Proximity Notification.wav', autoplay=True)

IndexError: list index out of range

# 9. Close imagej

In [72]:
ij.dispose()

In [73]:
ij = imagej.init('fiji\\fiji\\Fiji.app')